# Création du modèle

Pour ce projet de détection de language des signes nous allons utilisé l'objet détection API de TensorFlow avec SSD MobileNet v2 320x320

## Importation 

In [1]:
import numpy as np
import os
import pathlib
from io import StringIO

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2

from google.protobuf import text_format

## Setup Paths

on commence par setup les paths pour le projet pour que ce soit plus facile ensuite

### Model Name

In [2]:
# definition du nom de notre model
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

## Paths

In [3]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME +'/'
MODEL_EXPORTED_PATH = WORKSPACE_PATH + '/exported_model'

# Create Label Map

L'api a besoin d'une label map, qui va servir pour la labellisation

In [4]:
labels = [
    {'name':'A', 'id':1},
    {'name':'B', 'id':2},
    {'name':'C', 'id':3},
    {'name':'D', 'id':4},
    {'name':'E', 'id':5},
    {'name':'F', 'id':6},
    {'name':'G', 'id':7},
    {'name':'H', 'id':8},
    {'name':'I', 'id':9},
    {'name':'J', 'id':10},
    {'name':'K', 'id':11},
    {'name':'L', 'id':12},
    {'name':'M', 'id':13},
    {'name':'N', 'id':14},
    {'name':'O', 'id':15},
    {'name':'P', 'id':16},
    {'name':'Q', 'id':17},
    {'name':'R', 'id':18},
    {'name':'S', 'id':19},
    {'name':'T', 'id':20},
    {'name':'U', 'id':21},
    {'name':'V', 'id':22},
    {'name':'W', 'id':23},
    {'name':'X', 'id':24},
    {'name':'Y', 'id':25},
    {'name':'Z', 'id':26}
        ]
# création d'un ficher label_map.pbtxt dans le sous-dossier annotation du dossier workspace
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Create TF records

L'api a aussi besoin de ficher tfrecord pour son entrainement

In [5]:
# Creation de deux tfrecord, un pour train et un pour test
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## Download TF Models Pretrained Models from Tensorflow Model Zoo

Nous allons ensuite télécharger le github de tensorflow/models, nous aurons besoin de scripts se trouvant dans ce github

pour ce faire on peut soit git clone soit le télécharger à la main : https://github.com/tensorflow/models.git

In [19]:
# aller dans le dossier et git clone 
# cette celule est commentée car cette étape a été faite à la main
# !cd Tensorflow && git clone https://github.com/tensorflow/models

In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Copy Model Config to Training Folder

pour entrainer le model nous allons copier/coller le fichier de config du model choisi vers le dossier de notre model

In [10]:
# Cette celule est commentée par fait a la main

# Creation du dossier et copier/coller du fichier de configuration

#!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
#!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

'cp' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


## Update Config For Transfer Learning

Modification du fichier de configuration pour l'adapter a notre cas 

In [6]:
# Attribution du ficher a une variable
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [7]:
# affichage du fichier
config

{'model': ssd {
   num_classes: 26
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [8]:
# Lecture du fichier
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [9]:
# Modification du ficher
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 6
pipeline_config.train_config.fine_tune_checkpoint = CHECKPOINT_PATH + 'ckpt-7'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [10]:
# Ecriture sur le fichier
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## Train the model

L'entrainement d'un model avec l'api se fait en ligne de commande dans le terminal

In [11]:
# Print de la commande à coller dans le terminal pour entrainer le model
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


## Save model

Pareil que pour l'entrainement l'enregistrement se fait en ligne de commande dans le terminal

In [12]:
# print la commande coller dans le terminal pour exporter le model
print("""python {}/research/object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path {} --trained_checkpoint_dir {} --output_directory {}/{}""".format(APIMODEL_PATH, CONFIG_PATH, CHECKPOINT_PATH, MODEL_EXPORTED_PATH, CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type image_tensor --pipeline_config_path Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --trained_checkpoint_dir Tensorflow/workspace/models/my_ssd_mobnet/ --output_directory Tensorflow/workspace/exported_model/my_ssd_mobnet


## Evaluate the model

Pour voir les graphes de ``loss``` du modèle nous allons utiliser Tensorboard, toujours dans le terminal

In [13]:
print("""tensorboard --logdir={}""".format(CHECKPOINT_PATH))

tensorboard --logdir=Tensorflow/workspace/models/my_ssd_mobnet/


Voici les graphes des ``loss`` pour ce modèle avec les paramètres actuels


![classification loss](ressources\class_loss.jpg)

![localisation loss](ressources\loc_loss.jpg)

![normalized loss](ressources\norm_loss.jpg)

![regularization loss](ressources\reg_loss.jpg)

![total loss](ressources\tot_loss.jpg)



et voici celui du ``learning rate``

![learning rate](ressources\lr.jpg)